In [178]:
import pandas as pd
import numpy as np

df1 = pd.read_csv("MTJ_JSON_Dream Trawler.csv")
df2 = pd.read_csv("EDHREC Number of Decks.csv")
df3 = pd.read_csv("Commander Spellbook Database - combos.csv")

In [179]:
df_c = df3.merge(df2, how='right', left_on="ID", right_on="ID_spellbook")
df_c = df_c[~df_c['Card 1'].isna()] # Tirar linhas com NA

In [180]:
# Contar a frequência que as cartas aparecem no combo
cols = [x for x in df_c.columns if "Card" in x]
df_cards = pd.concat([df_c[c] for c in cols]).value_counts()

In [181]:
# Contar o número de decks que as cartas de combo aparecem
df_c2 = pd.DataFrame(df_cards, columns=['Frequency in decks']).reset_index().rename(columns={'index': 'Card'})
df_c2['Number of decks'] = 0

for i in range(1,10):
    n = df_c[cols+["n_of_decks"]].groupby("Card "+str(i)).sum()
    df_c2 = df_c2.merge(n, how='left', left_on='Card', right_on="Card "+str(i))
    df_c2['Number of decks'] = df_c2['Number of decks'] + df_c2['n_of_decks'].replace(np.NAN, 0)
    df_c2 = df_c2.drop('n_of_decks', axis=1)

In [182]:
# Correlação entre número de combos e quantidade de decks
df_c2[['Number of decks', 'Frequency in decks']].corr(method='pearson')

,Number of decks,Frequency in decks
Number of decks,1.000000,0.675387
Frequency in decks,0.675387,1.000000


In [183]:
# Cálculo da importancia relativa da carta (somatória do número de decks por número de cartas no combo)
df_c2['Relative importance'] = 0
df_c['n_of_cards'] = df_c[cols].notnull().sum(axis=1)

df_c['n_of_decks_2'] = df_c['n_of_decks'] / df_c['n_of_cards']

for i in range(1,10):
    n = df_c[cols+["n_of_decks_2"]].groupby("Card "+str(i)).sum()
    df_c2 = df_c2.merge(n, how='left', left_on='Card', right_on="Card "+str(i))
    df_c2['Relative importance'] = df_c2['Relative importance'] + df_c2['n_of_decks_2'].replace(np.NAN, 0)
    df_c2 = df_c2.drop('n_of_decks_2', axis=1)

In [184]:
# Cálculo da dificuldade do combo (número de sentenças nos pré-requisitos por número de cartas no combo)
df_c2['Relative difficulty of combo'] = 0

df_c['simp'] = (df_c['Prerequisites'] + ".").str.split('.').str.len()

for i in range(1,10):
    n = df_c[cols+["simp"]].groupby("Card "+str(i)).sum()
    df_c2 = df_c2.merge(n, how='left', left_on='Card', right_on="Card "+str(i))
    df_c2['Relative difficulty of combo'] = df_c2['Relative difficulty of combo'] + df_c2['simp'].replace(np.NAN, 0)
    df_c2 = df_c2.drop('simp', axis=1)

df_c2['Relative difficulty of combo'] = df_c2['Relative difficulty of combo'] / df_c2['Frequency in decks']

In [185]:
# Cálculo do índice da combagem (Coralheim) (importância relativa por dificuldade relativa)
df_c2["Índice Coralhelm"] = df_c2["Relative importance"] / df_c2["Relative difficulty of combo"]
df_c2.sort_values('Índice Coralhelm', ascending=False).head(20)

,Card,Frequency in decks,Number of decks,Relative importance,Relative difficulty of combo,Índice Coralhelm
0,Retreat to Coralhelm,1028,115881.0,29195.666667,3.266537,8937.803851
1,Ashnod's Altar,673,104011.0,31224.850000,4.193165,7446.606680
220,Dramatic Reversal,33,72742.0,29015.500000,5.454545,5319.508333
22,Isochron Scepter,162,77110.0,29938.166667,5.660494,5288.967285
811,Exquisite Blood,8,41950.0,20975.000000,4.125000,5084.848485
2,Phyrexian Altar,518,67588.0,20272.783333,4.096525,4948.775573
84,Sensei's Divining Top,73,51367.0,16546.000000,3.465753,4774.142292
56,Thassa's Oracle,90,57597.0,25427.045238,5.677778,4478.344562
97,Basalt Monolith,66,40986.0,18456.375000,4.303030,4289.157570
7,"Kiki-Jiki, Mirror Breaker",247,42128.0,18636.419048,4.611336,4041.435913


In [186]:
# Merge com dataset de cartas
df = df1.merge(df_c2, how='left', left_on='name', right_on='Card')

# Retirar cartas não válidas
list_not_valid = ["Ancestral Recall","Balance","Biorhythm","Black Lotus","Braids, Cabal Minion","Channel","Chaos Orb"
,"Coalition Victory","Emrakul, the Aeons Torn","Erayo, Soratami Ascendant","Falling Star","Fastbond","Flash","Gifts Ungiven"
,"Golos, Tireless Pilgrim","Griselbrand","Hullbreacher","Iona, Shield of Emeria","Karakas","Leovold, Emissary of Trest","Library of Alexandria","Limited Resources"
,"Lutri, the Spellchaser","Mox Emerald","Mox Jet","Mox Pearl","Mox Ruby","Mox Sapphire","Panoptic Mirror","Paradox Engine"
,"Primeval Titan","Prophet of Kruphix","Recurring Nightmare","Rofellos, Llanowar Emissary","Shahrazad","Sway of the Stars"
,"Sundering Titan","Sylvan Primordial","Time Vault","Time Walk","Tinker","Tolarian Academy","Trade Secrets","Upheaval"
,"Yawgmoth’s Bargain","Invoke Prejudice","Cleanse","Stone-Throwing Devils","Pradesh Gypsies","Jihad","Imprison","Crusade"
,"Amulet of Quoz","Bronze Tablet","Contract from Below","Darkpact","Demonic Attorney","Jeweled Bird","Rebirth","Tempest Efreet"
,"Timmerian Friends","Adriana's Valor","Advantageous Proclamation","Assemble the Rank and Vile","Backup Plan","Brago's Favor"
,"Double Stroke","Echoing Boon","Emissary's Ploy","Hired Heist","Hold the Perimeter","Hymn of the Wilds","Immediate Action"
,"Incendiary Dissent","Iterative Analysis","Muzzio's Preparations","Natural Unity","Power Play","Secret Summoning","Secrets of Paradise"
,"Sentinel Dispatch","Sovereign's Realm","Summoner's Bound","Unexpected Potential","Weight Advantage","Worldknit"]

df = df[~df['name'].isin(list_not_valid)]

# Retirar sets não válidos
list_not_sanctioned = ['CED', 'UGL', 'UNH', 'UST']

df = df[~df['setCode'].isin(list_not_sanctioned)]

In [187]:
# Correções de texto

## colorIdentity
df['s'] = df['colorIdentity'].str.split(r"[^a-zA-Z]*")

def get_letters(row):
    n = ''.join([x for x in row if x.isalpha()])
    return n if n != "" else 'C'

df['colorStr'] = df['s'].apply(get_letters)

## manaCost
df['manaCostStr'] = df['manaCost'].str.replace("{", "").str.replace("}", "")

## supertypes, subtypes e types
df['superT'] = df['supertypes'].str.replace(";", " ")
df['subT'] = df['subtypes'].str.replace(";", " ")
df['typesStr'] = df['types'].str.replace(";", " ")

## keywords
df['k'] = df['keywords'].str.split(r"[^a-zA-Z]+")

def get_letters(row):
    n = ';'.join([x for x in row if x.isalpha()])
    return n

df['keys'] = df['k'].apply(get_letters)

/tmp/ipykernel_36322/1183593895.py:13: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df['manaCostStr'] = df['manaCost'].str.replace("{", "").str.replace("}", "")


In [188]:
cols = ['name', 'completeName', 'setCode', 'colorStr', 'convertedManaCost', 'text', 'manaCostStr', 'power', 'toughness', 'rarity', 'superT', 'subT', 'typesStr', 'keys', 'loyalty']
df = df[cols]
df

,name,completeName,setCode,colorStr,convertedManaCost,text,manaCostStr,power,toughness,rarity,superT,subT,typesStr,keys,loyalty
0,Abuna's Chant,Abuna's Chant,5DN,W,4,Choose one —\n• You gain 5 life.\n• Prevent th...,3W,NaN,NaN,common,NaN,NaN,Instant,Entwine,-1
1,Armed Response,Armed Response,5DN,W,3,Armed Response deals damage to target attackin...,2W,NaN,NaN,common,NaN,NaN,Instant,,-1
2,Auriok Champion,Auriok Champion,5DN,W,2,Protection from black and from red\nWhenever a...,WW,1,1,rare,NaN,Human Cleric,Creature,Protection,-1
3,Auriok Salvagers,Auriok Salvagers,5DN,W,4,{1}{W}: Return target artifact card with mana ...,3W,2,4,uncommon,NaN,Human Soldier,Creature,,-1
4,Auriok Windwalker,Auriok Windwalker,5DN,W,4,Flying\n{T}: Attach target Equipment you contr...,3W,2,3,rare,NaN,Human Wizard,Creature,Flying,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24100,Lavaglide Pathway,Riverglide Pathway // Lavaglide Pathway,ZNR,RU,0,{T}: Add {R}.,NaN,NaN,NaN,rare,NaN,NaN,Land,,-1
24101,Throne of Makindi,Throne of Makindi,ZNR,C,0,"{T}: Add {C}.\n{1}, {T}: Put a charge counter ...",NaN,NaN,NaN,rare,NaN,NaN,Land,,-1
24102,"A-Omnath, Locus of Creation","A-Omnath, Locus of Creation",ZNR,GRUW,5,"When Omnath, Locus of Creation enters the batt...",1RGWU,4,4,mythic,Legendary,Elemental,Creature,Landfall;Scry,-1
24103,"A-Phylath, World Sculptor","A-Phylath, World Sculptor",ZNR,GR,6,"Trample\nWhen Phylath, World Sculptor enters t...",4RG,5,5,rare,Legendary,Elemental,Creature,Landfall;Trample,-1


In [190]:
df.to_csv('Índice Coralheim.csv')